In [2]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
import os
os.chdir('..')

# LocalFile 持久化

In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_zhipu import ChatZhipuAI
from langchain_core.output_parsers import StrOutputParser

# 定义chain
model = ChatZhipuAI()
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一个数学老师。"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)
chain = prompt | model | StrOutputParser()

In [4]:
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_chinese.memory.history import LocalFileMessageHistory
from langchain_chinese import MemoryManager, WithMemoryBinding, create_session_id

# 定义记忆体
memory = MemoryManager(
    lambda session_id: LocalFileMessageHistory(session_id),
    shorterm_memory = ConversationBufferWindowMemory(return_messages=True, k=2)
)

# 记忆绑定管理
withMemoryChain = WithMemoryBinding(
    chain,
    memory,
    input_messages_key="input",
    history_messages_key="history",
)

In [5]:
session_id = create_session_id()
session_id

'2024_04_28_204311_default_8406'

In [6]:
# 调用
withMemoryChain.invoke(
    {"input": "方程式是什么意思?"},
    config={"configurable": {"session_id": session_id}}
)

'方程式是一个数学表达式，它表示两个表达式的值相等，通常包含一个或多个未知数，用等号“=”连接。在方程式中，未知数通常用字母（如x、y）来表示，目的是要找出这些未知数的值，使得等号两边的表达式相等。\n\n方程式的一般形式如下：\n\n\\[ A(x) = B(x) \\]\n\n其中，\\( A(x) \\) 和 \\( B(x) \\) 是数学表达式，可以包含数字、变量和运算符。方程式的解是指使得等号两边相等的未知数的值。\n\n例如，下面是一个简单的线性方程：\n\n\\[ 2x + 3 = 7 \\]\n\n这个方程的解是 \\( x = 2 \\)，因为当 \\( x \\) 等于2时，等号两边的值都是7。'

In [7]:
# 调用
withMemoryChain.invoke(
    {"input": "你可以用小学生二年级语言给我解释吗?"},
    config={"configurable": {"session_id": session_id}}
)

'当然可以！想象一下你有一袋苹果，如果我把苹果分给你和其他小朋友，然后我告诉你总共分了多少个苹果，你需要找出每个小朋友分到了几个苹果。\n\n方程式就像是一个小谜题，它说的是：“这些苹果分成几份后，每份有几个苹果？”比如，如果我们这样写：\n\n\\[ 2x + 3 = 7 \\]\n\n这个方程式可以这样说：“如果你把每份苹果的数量叫做‘x’，然后你拿了两份，再加上三个苹果，一共是7个苹果。那么，每份有几个苹果呢？”\n\n在这个例子中，我们要找出“x”是多少。我们可以用简单的步骤来解决这个谜题：\n\n1. 先从7个苹果中减去那额外的3个苹果，因为那3个苹果不是我们要找的“每份”的苹果。\n   \\( 7 - 3 = 4 \\)\n\n2. 现在我们知道，剩下的4个苹果是两份的总量。所以我们只需要把4个苹果平均分给两份。\n   \\( 4 ÷ 2 = 2 \\)\n\n所以，每份苹果就是2个。这就是“x”的值，也就是我们的答案。每个小朋友分到了2个苹果。'

In [8]:
# 调用
withMemoryChain.invoke(
    {"input": "那谢谢哦"},
    config={"configurable": {"session_id": session_id}}
)

'不客气！如果你有其他问题或者需要帮助，随时欢迎来问。祝你学习愉快！'

In [9]:
for chunk in withMemoryChain.stream(
    {"input": "我最初的问题是什么来着?"},
    config={"configurable": {"session_id": session_id}},
):
    print(chunk, end="|", flush=True)

你|最初|的问题是|：“|你|是一个|数学|老师|。”| 你|在|设定|一个|角色|，|好像|我是|你的|数学|老师|，|准备好|回答|你的|数学|问题|或者|解释|数学|概念|。|如果你|有|其他|数学|相关|的问题|或者|需要|进一步的|解释|，|请|随时|告诉我|。||

In [10]:
memory.shorterm_messages(session_id)

[HumanMessage(content='那谢谢哦'),
 AIMessage(content='不客气！如果你有其他问题或者需要帮助，随时欢迎来问。祝你学习愉快！'),
 HumanMessage(content='我最初的问题是什么来着?'),
 AIMessage(content='你最初的问题是：“你是一个数学老师。” 你在设定一个角色，好像我是你的数学老师，准备好回答你的数学问题或者解释数学概念。如果你有其他数学相关的问题或者需要进一步的解释，请随时告诉我。')]

In [11]:
memory.longterm_messages(session_id)

[HumanMessage(content='方程式是什么意思?'),
 AIMessage(content='方程式是一个数学表达式，它表示两个表达式的值相等，通常包含一个或多个未知数，用等号“=”连接。在方程式中，未知数通常用字母（如x、y）来表示，目的是要找出这些未知数的值，使得等号两边的表达式相等。\n\n方程式的一般形式如下：\n\n\\[ A(x) = B(x) \\]\n\n其中，\\( A(x) \\) 和 \\( B(x) \\) 是数学表达式，可以包含数字、变量和运算符。方程式的解是指使得等号两边相等的未知数的值。\n\n例如，下面是一个简单的线性方程：\n\n\\[ 2x + 3 = 7 \\]\n\n这个方程的解是 \\( x = 2 \\)，因为当 \\( x \\) 等于2时，等号两边的值都是7。'),
 HumanMessage(content='你可以用小学生二年级语言给我解释吗?'),
 AIMessage(content='当然可以！想象一下你有一袋苹果，如果我把苹果分给你和其他小朋友，然后我告诉你总共分了多少个苹果，你需要找出每个小朋友分到了几个苹果。\n\n方程式就像是一个小谜题，它说的是：“这些苹果分成几份后，每份有几个苹果？”比如，如果我们这样写：\n\n\\[ 2x + 3 = 7 \\]\n\n这个方程式可以这样说：“如果你把每份苹果的数量叫做‘x’，然后你拿了两份，再加上三个苹果，一共是7个苹果。那么，每份有几个苹果呢？”\n\n在这个例子中，我们要找出“x”是多少。我们可以用简单的步骤来解决这个谜题：\n\n1. 先从7个苹果中减去那额外的3个苹果，因为那3个苹果不是我们要找的“每份”的苹果。\n   \\( 7 - 3 = 4 \\)\n\n2. 现在我们知道，剩下的4个苹果是两份的总量。所以我们只需要把4个苹果平均分给两份。\n   \\( 4 ÷ 2 = 2 \\)\n\n所以，每份苹果就是2个。这就是“x”的值，也就是我们的答案。每个小朋友分到了2个苹果。'),
 HumanMessage(content='那谢谢哦'),
 AIMessage(content='不客气！如果你有其他问题或者需要帮助，随时欢迎来问。祝你学习愉快！'),
 HumanMessage(content='我最初的问题是什么来着?'

# redis 持久化

```shell
pip install -U langchain-community redis
```

创建窗口管理器时要指定redis的工厂函数，其他与上面的例子相同。

In [ ]:
from langchain_chinese import MemoryManager
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.memory import ConversationBufferMemory, ConversationBufferWindowMemory
from langchain_community.chat_message_histories import RedisChatMessageHistory

window = ConversationBufferWindowMemory(
    return_messages=True, k=2, chat_memory = ChatMessageHistory()
)

memory = MemoryManager(
    lambda session_id: RedisChatMessageHistory(
        session_id, url="redis://localhost:6379"
    ),
    shorterm_memory = window
)